In [69]:
import solid as sp
import numpy as np
from solid.utils import *

def bezier_curve(P0, P1, P2, t_values):
    P = np.zeros((len(t_values), 2))
    for i, t in enumerate(t_values):
        P[i] = (1-t)**2 * P0 + 2*(1-t)*t * P1 + t**2 * P2
    return P

def generate_earpiece(P0, P1, P2, t_values, frame_thickness, text, text_size, text_depth, text_offset):
    curve_points = bezier_curve(P0, P1, P2, t_values)
    temple_poly = sp.polygon(points=curve_points.tolist())
    temple_3d = sp.linear_extrude(height=frame_thickness)(temple_poly)
    engraving = sp.linear_extrude(height=text_depth)(sp.text(text, size=text_size))
    earpiece = sp.difference()(
        temple_3d,
        sp.translate([text_offset[0], text_offset[1], frame_thickness - text_depth])(engraving)
    )
    return earpiece

def glasses_frame(frame_width, frame_height, frame_thickness, bridge_width, bridge_curve, left_earpiece, right_earpiece):
    eyepiece = sp.scale([frame_width / 100.0, frame_height / 100.0, 1])(sp.cylinder(h=frame_thickness, r=50))
    bridge = sp.cylinder(h=frame_thickness, r=bridge_width / 2)
    bridge_cut = sp.translate([0, -bridge_curve, 0])(sp.cylinder(h=frame_thickness, r=bridge_width))
    bridge = sp.difference()(bridge, bridge_cut)

    cutout_radius_x = (frame_width / 2) * 0.9  
    cutout_radius_y = (frame_height / 2) * 0.9  
    cutout = sp.scale([cutout_radius_x / 50.0, cutout_radius_y / 50.0, 1])(sp.cylinder(h=frame_thickness + 1, r=50))

    frame = sp.difference()(
        sp.union()(
            sp.translate([-frame_width / 2, 0, 0])(eyepiece),
            sp.translate([frame_width / 2, 0, 0])(eyepiece),
            sp.translate([0, frame_height / 5, 0])(bridge),
            left_earpiece,
            right_earpiece
        ),
        sp.translate([-frame_width / 2, 0, 0])(cutout),
        sp.translate([frame_width / 2, 0, 0])(cutout)
    )
    return frame


frame_width = 60
frame_height = 45
frame_thickness = 5
bridge_width = 20
bridge_curve = 20
earpiece_length = 100
earpiece_start_y = frame_height * 0.4
text_depth = 0.5  
text_size = 10    
text_offset = [earpiece_length * 0.2, 20]  

P0 = np.array([0, 0])
P1 = np.array([earpiece_length * 0.3, 20])
P2 = np.array([earpiece_length, -20])
t_values = np.linspace(0, 1, 100)

left_earpiece_x_translation = -frame_width / 2 - frame_thickness * 2 - 10
right_earpiece_x_translation = frame_width / 2 + frame_thickness * 2 + 10
earpiece_z_translation = frame_thickness * 0.5

left_earpiece = sp.translate([left_earpiece_x_translation, earpiece_start_y, earpiece_z_translation])(
    sp.rotate([90, 90, 90])(
        generate_earpiece(P0, P1, P2, t_values, frame_thickness, "Dinking Buddies", text_size, text_depth, text_offset)
    )
)

right_earpiece = sp.translate([right_earpiece_x_translation, earpiece_start_y, earpiece_z_translation])(
    sp.rotate([90, 90, 90])(
        generate_earpiece(P0, P1, P2, t_values, frame_thickness, "Dinking Buddies", text_size, text_depth, text_offset)
    )
)

model = glasses_frame(frame_width, frame_height, frame_thickness, bridge_width, bridge_curve, left_earpiece, right_earpiece)

scad_code = sp.scad_render(model)
with open('v1_glasses.scad', 'w') as f:
    f.write(scad_code)